# Load Data

## Import Libraries

In [ ]:
# Import pandas, numpy, and matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# seaborn is a data visualization library built on matplotlib
import seaborn as sns

# set the plotting style
sns.set_style("whitegrid")

## Load Diabetes Datasets

### Diabetes 130-US Hospitals for Years 1999-2008

In [ ]:
# Loading dataset
diabetic_UCI = pd.read_csv("https://raw.githubusercontent.com/data5100-group/diabetes-risk-analysis/main/data/diabetic_data.csv")

# Loading mapping
ids_mapping_UCI = pd.read_csv("https://raw.githubusercontent.com/data5100-group/diabetes-risk-analysis/main/data/IDS_mapping.csv")

### Comprehensive Diabetes Clinical Dataset

In [ ]:
# Loading dataset
diabetes_kaggle = pd.read_csv("https://raw.githubusercontent.com/data5100-group/diabetes-risk-analysis/main/data/diabetes_dataset.csv")

## Understanding the Data

In [ ]:
diabetes_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   year                  100000 non-null  int64  
 1   gender                100000 non-null  object 
 2   age                   100000 non-null  float64
 3   location              100000 non-null  object 
 4   race:AfricanAmerican  100000 non-null  int64  
 5   race:Asian            100000 non-null  int64  
 6   race:Caucasian        100000 non-null  int64  
 7   race:Hispanic         100000 non-null  int64  
 8   race:Other            100000 non-null  int64  
 9   hypertension          100000 non-null  int64  
 10  heart_disease         100000 non-null  int64  
 11  smoking_history       100000 non-null  object 
 12  bmi                   100000 non-null  float64
 13  hbA1c_level           100000 non-null  float64
 14  blood_glucose_level   100000 non-null  int64  
 15  d

In [ ]:
diabetic_UCI.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 50 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  payer_code                101766 non-null  object
 11  medical_specialty         101766 non-null  object
 12  num_lab_procedures        101766 non-null  int64 
 13  num_procedures            101766 non-null  int64 
 14  num_

# Clean the Datasets

## Columns to Keep

In [ ]:
diabetic_UCI = diabetic_UCI[[
    # Identifiers: One patient number can have multiple encounter ids
   "encounter_id", "patient_nbr",

    # Demographics
    "race", "gender", "age", "weight",

    # Admission/Discharge info
    "admission_type_id", "discharge_disposition_id", "admission_source_id",

    # Hospital Stay/ Utilization
    "time_in_hospital", "num_medications",
    "number_outpatient", "number_emergency", "number_inpatient",

    # Diagnosis Codes
    "diag_1", "diag_2", "diag_3", "number_diagnoses",

    # Medication Information
    "metformin", "repaglinide", "nateglinide", "chlorpropamide", "glimepiride",
    "acetohexamide", "glipizide", "glyburide", "tolbutamide", "pioglitazone",
    "rosiglitazone", "acarbose", "miglitol", "troglitazone", "tolazamide", "examide",
    "citoglipton", "insulin", "glyburide-metformin", "glipizide-metformin", "glimepiride-pioglitazone",
    "metformin-rosiglitazone", "metformin-pioglitazone", "change", "diabetesMed",

    # Outcome
    "readmitted"
]]

In [ ]:
diabetes_kaggle = diabetes_kaggle[[
    # Demographics
    "age", "gender", "race:AfricanAmerican", "race:Asian", "race:Caucasian", "race:Other",

    # Factors
    "hypertension", "heart_disease", "smoking_history", "bmi", "hbA1c_level", "blood_glucose_level",

    # Outcome
    "diabetes"
]]

## Join diabetic_UCI and ids_mapping_UCI

In [ ]:
# Make sure the join key is numeric in both
diabetic_UCI["admission_type_id"] = pd.to_numeric(diabetic_UCI["admission_type_id"], errors="coerce")
ids_mapping_UCI["admission_type_id"] = pd.to_numeric(ids_mapping_UCI["admission_type_id"], errors="coerce")

In [ ]:
# Left-merge to attach the label then tidy
diabetic_UCI_merged = (
    diabetic_UCI.merge(ids_mapping_UCI, on="admission_type_id", how="left")
                .rename(columns={"description": "admission_type"})
)
diabetic_UCI_merged["admission_type"] = diabetic_UCI_merged["admission_type"].astype("category")

# Delete duplicated data from join
ids_mapping_dedup = ids_mapping_UCI.drop_duplicates("admission_type_id")
diabetic_UCI_merged = diabetic_UCI.merge(
    ids_mapping_dedup, on="admission_type_id", how="left", validate="many_to_one"
).rename(columns={"description":"admission_type"})

In [ ]:
diabetic_UCI_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101766 entries, 0 to 101765
Data columns (total 45 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0   encounter_id              101766 non-null  int64 
 1   patient_nbr               101766 non-null  int64 
 2   race                      101766 non-null  object
 3   gender                    101766 non-null  object
 4   age                       101766 non-null  object
 5   weight                    101766 non-null  object
 6   admission_type_id         101766 non-null  int64 
 7   discharge_disposition_id  101766 non-null  int64 
 8   admission_source_id       101766 non-null  int64 
 9   time_in_hospital          101766 non-null  int64 
 10  num_medications           101766 non-null  int64 
 11  number_outpatient         101766 non-null  int64 
 12  number_emergency          101766 non-null  int64 
 13  number_inpatient          101766 non-null  int64 
 14  diag

In [ ]:
diabetes_kaggle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   age                   100000 non-null  float64
 1   gender                100000 non-null  object 
 2   race:AfricanAmerican  100000 non-null  int64  
 3   race:Asian            100000 non-null  int64  
 4   race:Caucasian        100000 non-null  int64  
 5   race:Other            100000 non-null  int64  
 6   hypertension          100000 non-null  int64  
 7   heart_disease         100000 non-null  int64  
 8   smoking_history       100000 non-null  object 
 9   bmi                   100000 non-null  float64
 10  hbA1c_level           100000 non-null  float64
 11  blood_glucose_level   100000 non-null  int64  
 12  diabetes              100000 non-null  int64  
dtypes: float64(3), int64(8), object(2)
memory usage: 9.9+ MB


# Exporting CSVs

In [ ]:
# Turn the cleaned datasets to CSVs

diabetic_UCI_merged.to_csv(
    'diabetic_UCI_merged',
    encoding='utf-8-sig',
    index=False
)

diabetes_kaggle.to_csv(
    'diabetes_kaggle',
    encoding='utf-8-sig',
    index=False
)

In [ ]:
# Download datasets
from google.colab import files

files.download('diabetic_UCI_merged')
files.download('diabetes_kaggle')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##